In [7]:
import sqlite3

# Connect to SQLite DB (creates file if it doesn't exist)
conn = sqlite3.connect('ecommerce_trackers.db')
cursor = conn.cursor()

# Create Customers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customers (
    CustomerID INTEGER PRIMARY KEY,
    Name TEXT,
    Email TEXT
)
''')

# Create Products table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT,
    Price REAL,
    Stock INTEGER
)
''')

# Create Orders table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    OrderID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    ProductID INTEGER,
    Quantity INTEGER,
    OrderDate TEXT,
    FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY(ProductID) REFERENCES Products(ProductID)
)
''')

# Insert data (ensure matching IDs for correct queries)
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (1, 'Amit', 'amit@example.com')")
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (3, 'Shubham', 'shubhu@example.com')")

cursor.execute("INSERT OR IGNORE INTO Products VALUES (101, 'Keyboard', 899.00, 10)")
cursor.execute("INSERT OR IGNORE INTO Products VALUES (102, 'Wireless Mouse', 499.99, 3)")  # low stock item

cursor.execute("INSERT OR IGNORE INTO Orders VALUES (202, 1, 101, 2, '2025-04-30')")
cursor.execute("INSERT OR IGNORE INTO Orders VALUES (203, 3, 102, 1, '2025-04-28')")

# Commit the inserts
conn.commit()

# Query 1: Customer order history
print("\n📜 Customer Order History:")
cursor.execute('''
SELECT C.Name, P.ProductName, O.Quantity, O.OrderDate
FROM Orders O
JOIN Customers C ON O.CustomerID = C.CustomerID
JOIN Products P ON O.ProductID = P.ProductID
WHERE C.CustomerID = 1
''')
rows = cursor.fetchall()
if rows:
    for row in rows:
        print(row)
else:
    print("No orders found for this customer.")

# Query 2: Total purchase value per customer
print("\n Total Purchase Value:")
cursor.execute('''
SELECT C.Name, SUM(P.Price * O.Quantity) AS TotalSpent
FROM Orders O
JOIN Customers C ON O.CustomerID = C.CustomerID
JOIN Products P ON O.ProductID = P.ProductID
GROUP BY C.Name
''')
for row in cursor.fetchall():
    print(f"{row[0]} spent ₹{row[1]:.2f}")

# Query 3: Low-stock alerts
print("\n Low Stock Alerts:")
cursor.execute('''
SELECT ProductName, Stock
FROM Products
WHERE Stock < 5
''')
low_stock = cursor.fetchall()
if low_stock:
    for row in low_stock:
        print(f"Low Stock: {row[0]} has only {row[1]} left.")
else:
    print("All products are sufficiently stocked.")

# Close the connection




📜 Customer Order History:
('Amit', 'Keyboard', 2, '2025-04-30')

 Total Purchase Value:
Amit spent ₹1798.00
Shubham spent ₹499.99

 Low Stock Alerts:
Low Stock: Wireless Mouse has only 3 left.


In [5]:
conn.commit()
conn.close()